In [1]:
import librosa
import librosa.display
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [3]:
DATA_PATH = 'C:/Users/ppeng/Documents/AI_babyclassified/donateacry_corpus_cleaned_and_updated_data/'

In [4]:
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


In [5]:
labels, _, _ = get_labels(DATA_PATH)
data = {}
for label in labels:
      data[label] = {}
      data[label]['path'] = [DATA_PATH  + label + '/' +  wavfile for wavfile in os.listdir(DATA_PATH + '/' + label)]
      D = []
      for wavfile in data[label]['path']:
        y, sr = librosa.load(wavfile , duration = 4.97)  
        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        D.append(ps)
      data[label]['melspec'] = D
print(labels)

['animal', 'belly_pain', 'burping', 'discomfort', 'Exteriorurban noises', 'Human, non-speech sounds', 'hungry', 'Interiordomestic sounds', 'Natural soundscapes & water sounds', 'tired']


In [6]:
print(labels)
print(len(labels))

['animal', 'belly_pain', 'burping', 'discomfort', 'Exteriorurban noises', 'Human, non-speech sounds', 'hungry', 'Interiordomestic sounds', 'Natural soundscapes & water sounds', 'tired']
10


In [7]:
Data = []
for key in data:
    
      for melspec in data[key]['melspec']:
            Data.append((key,melspec))
data[key]['melspec'] = Data
print("Number of samples: ", len(Data))

Number of samples:  4841


In [8]:
import random
print("Number of samples: ", len(Data))
random.shuffle(Data)

train = Data[:3100]
test = Data[3100:]

y_train, X_train = zip(*train)
y_test, X_test = zip(*test)

Number of samples:  4841


In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

X_train_T = np.array([x.reshape( (128, 215, 1) ) for x in X_train])
X_test_T = np.array([x.reshape( (128, 215, 1) ) for x in X_test])

y_train_T = np.array(keras.utils.to_categorical(label_encoder.fit_transform(np.array(y_train)),10))
y_test_T = np.array(keras.utils.to_categorical(label_encoder.fit_transform(np.array(y_test)),10))


In [47]:
from keras.layers import BatchNormalization
model = Sequential()
input_shapeP=(128, 215, 1)
model.add(Conv2D(24,(5,5),strides=(1,1),input_shape=input_shapeP, activation='relu'))
model.add(MaxPooling2D((4, 2),strides=(4,2)))

model.add(Conv2D(48,(5,5),padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((4, 2),strides=(4,2)))

model.add(Conv2D(48,(5,5),padding='valid', activation='relu'))

model.add(Flatten())
model.add(Dropout(0.50))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(10,activation='softmax'))

In [48]:
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [69]:
model.fit(X_train_T,y_train_T,epochs=12,batch_size=128,validation_data=(X_test_T,y_test_T))

Train on 3100 samples, validate on 1741 samples
Epoch 1/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.8638 - accuracy: 0.6787 - val_loss: 1.6409 - val_accuracy: 0.6427
Epoch 2/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.9443 - accuracy: 0.6484 - val_loss: 1.7559 - val_accuracy: 0.6169
Epoch 3/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.9041 - accuracy: 0.6542 - val_loss: 1.6039 - val_accuracy: 0.6232
Epoch 4/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.9159 - accuracy: 0.6519 - val_loss: 1.6684 - val_accuracy: 0.6232
Epoch 5/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.8935 - accuracy: 0.6794 - val_loss: 1.5630 - val_accuracy: 0.6025
Epoch 6/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.8453 - accuracy: 0.6894 - val_loss: 1.8459 - val_accuracy: 0.6272
Epoch 7/12
3100/3100 [==============================] - 4s 1ms/step - loss: 0.8119 - acc

In [70]:
model.save('C:/Users/ppeng/Documents/AI_babyclassified/model_DataAugmented_ver42') 

In [3]:
model_ver1= keras.models.load_model('C:/Users/ppeng/Documents/AI_babyclassified/model_DataAugmented_ver21')

In [72]:
model_ver1.evaluate(X_test_T, y_test_T, verbose=0)

[1.8069568982006556, 0.6168868541717529]

In [ ]:
model_ver1.predict_classes()

In [45]:
model_ver1.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_119 (Conv2D)          (None, 124, 211, 24)      624       
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 31, 105, 24)       0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 27, 101, 48)       28848     
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 6, 50, 48)         0         
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 2, 46, 48)         57648     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4416)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4416)            

In [ ]:
print(X_test_T[1])
print(y_test_T[1])

In [ ]:
print(test[1])

In [ ]:
data[key]['melspec']